In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [4]:
# Подгрузим наши данные из соревнования

DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

In [5]:
df_train.info()

In [6]:
df_train.head(2)

In [7]:
df_test.info()

In [8]:
df_test.head(2)

In [9]:
sample_submission.head(2)

In [10]:
sample_submission.info()

In [11]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [12]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.corr(),annot=True)

In [13]:
#    nltk используем для оценки текста комментария
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.downloader.download('vader_lexicon')

In [14]:
%%time

#    Используем Counter для подсчета популярных тегов средствами Python, а не Pandas
#    LabelEncoder используем для передачи номинальных категориальных признаков в модель напрямую

from collections import Counter
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder


#Поскольку отель встречается несколько раз, делаем из него номинальный признак
data['hotel_name'] = LabelEncoder().fit_transform(data['hotel_name'])

#Отели расположены в 5 городах, используем это
data['hotel_location'] = data['hotel_address'].apply(lambda s: ' '.join((s.split(' ')[-2:])))
data.drop('hotel_address',axis = 1, inplace = True)

#    Преобразуем координаты следующим образом:
#    Расчитаем минимальное, максимальное и среднее значение широты и долготы, для каждого города.
#    Заменим пропуски на среднее значение
#    Выполним операцию MinMaxScaler, с учетом каждого города 
#    вычтем из координаты отеля минимальную широту и долготу соответственно для его города
#    разделим на разницу между максимальным и минимальным значение координаты
#    Значения широта и долгота примут значения от 0 до 1 для каждого города

average_coords = dict(data.groupby('hotel_location')[['lat','lng']].agg(['min','mean','max']))
def align_coord_to_center(row,key,avg):
    value = row[key]
    if np.isnan(row[key]):
        value = avg[(key,'mean')][row['hotel_location']]
    result = (value - avg[(key,'min')][row['hotel_location']]) / (avg[(key,'max')][row['hotel_location']] - avg[(key,'min')][row['hotel_location']])
    return result

data['lat'] = data.apply(align_coord_to_center,axis = 1,key = 'lat',avg = average_coords)
data['lng'] = data.apply(align_coord_to_center,axis = 1,key = 'lng',avg = average_coords)

#Преобразуем город отеля в бинарные признаки методом OneHotEncoding, удалим исходный признак
data = pd.concat([data,pd.get_dummies(data['hotel_location'], drop_first = True)],axis=1)
data.drop('hotel_location',axis = 1, inplace = True)

#Оставим признак - возраст комментария в числовом виде для передачи в модель
data['days_since_review'] = data['days_since_review'].apply(lambda s: s.split(' ')[0]).astype('int')

#Из даты оставит также месяц и день недели
data['review_date'] = pd.to_datetime(data['review_date'])
data['month'] = pd.to_datetime(data['review_date']).dt.month
data['weekday'] = pd.to_datetime(data['review_date']).dt.weekday
data.drop('review_date',axis = 1, inplace = True)

#Из множества национальностей автора отзыва оставим наиболее популярные, остальные соберем в группу Other и закодируем методов OneHotEncoding
allowed_nationalities = ['United Kingdom', 'United States of America', 'Australia', 'Ireland']
data = pd.concat([data,pd.get_dummies(data['reviewer_nationality'].str.strip().apply(lambda s: s if s in allowed_nationalities else 'Other'),prefix = 'From ',drop_first=True)], axis = 1)
data.drop('reviewer_nationality',axis = 1, inplace = True)


#    Поработаем с текстом комментариев:
#    Видимо при заполнении отзыва предлагалось два поля: "Хорошее" и "Плохое"
#    И люди иногда писали "ничего" в одно из полей
#    Поскольку мы будем полагаться исключительно на размер комментария, нужно избавиться от таких случаев
#    Наиболее частые представлены в словаре, затрём их для обоих полей
#    Затем пересчитаем поля "количество слов в плохих/хороших комментариях"
#    Также создадим новый признак - разница между размеров комментариев
#    Дополнительно оценим полярность комментариев с помощью пакета nltk
#    Просуммируем результаты оценки обоих текстов

replace = {    
    'positive_review': ['no positive','nothing'],
    'negative_review': [
        'nothing really','no negative','nothing','n a','none','nothing at all','nothing to dislike',
        'everything was perfect','na','can t think of anything','nil','everything was great','absolutely nothing',
        'nothing to complain about','no','nothing not to like','nothing all good','no complaints','i liked everything'
        ,'liked everything'
    ]
    
}

for key in replace:
    for value in replace[key]:
        data.loc[data[key].str.strip().str.lower()==value,key] = ''
        
data['review_total_positive_word_counts'] = data['positive_review'].apply(lambda s: len(s.strip().split(' ')))
data['review_total_negative_word_counts'] = data['negative_review'].apply(lambda s: len(s.strip().split(' ')))

data['review_diff'] = data['positive_review'].str.len() - data['negative_review'].str.len()

polarity_columns = ['neg','neu','pos','compound']
analyzer = SentimentIntensityAnalyzer()

def get_polarity(row, analyzer):
    positive_counter = Counter(analyzer.polarity_scores(row['positive_review']))
    negative_counter = Counter(analyzer.polarity_scores(row['negative_review']))
    result_counter = positive_counter + negative_counter
    return [result_counter[col] for col in polarity_columns]

polarities = list(data.apply(get_polarity, analyzer = analyzer, axis = 1))

data_polarity = pd.DataFrame(polarities,columns = ['neg','neu','pos','compound'])

data = pd.concat([data, data_polarity], axis = 1)

data.drop(['positive_review','negative_review'],axis = 1, inplace = True)


#Из всего набора тегов оставим 10 наиболее популярных в виде бинарных признаков
TAGS_NUMBER = 11
c = Counter()
for tags in data['tags'].apply(lambda s: [x.strip() for x in s[1:-1].replace("'",'').split(',')]):
    for tag in tags:
        c[tag] += 1
for tag , _ in c.most_common(TAGS_NUMBER):
    tag_name = tag.lower().replace(' ','_')
    data[f"tag_{tag_name}"] = data['tags'].apply(lambda tags: int(tag in tags))        
data.drop(['tags','tag_leisure_trip'],axis = 1, inplace = True)

In [15]:
#    Для сравнения степени влияния признаков на целевую переменную воспользуемся тестами
#    Хи-квадрат для категориальных переменных
#    ANOVA для числовых переменных
#    Дальнейшее удаление признаков приводит к ухудшению итогового значения MAPE

from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif

cat_cols = [
    'Barcelona Spain','Milan Italy','Paris France','United Kingdom','Vienna Austria','month','weekday',
    'From _Ireland','From _Other','From _United Kingdom','From _United States of America',
    'tag_submitted_from_a_mobile_device','tag_couple','tag_stayed_1_night','tag_stayed_2_nights','tag_solo_traveler',
    'tag_stayed_3_nights','tag_business_trip','tag_group','tag_family_with_young_children','tag_stayed_4_nights'
]
num_cols = [
    'additional_number_of_scoring','average_score','review_total_negative_word_counts','total_number_of_reviews',
    'review_total_positive_word_counts','total_number_of_reviews_reviewer_has_given','days_since_review','lat','lng',
    'review_diff'
]

In [16]:
pd.Series(chi2(data[cat_cols],data['reviewer_score'].astype('int'))[0],index=cat_cols).sort_values(ascending=True).plot(kind='barh')

In [17]:
pd.Series(f_classif(data[num_cols],data['reviewer_score'].astype('int'))[0],index=num_cols).sort_values(ascending=True).plot(kind='barh')

In [18]:
data.nunique(dropna=False)

In [19]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(), annot=True)

In [20]:
data.columns

In [21]:
# убираем признаки которые еще не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis = 1, inplace=True)

In [22]:
data.info()

In [23]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [24]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [25]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [26]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [27]:
%%time

# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)
#model = RandomForestRegressor(n_estimators=20, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [28]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [29]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAPE:', metrics.mean_absolute_error(y_test, y_pred))
#MAPE: 0.8785584855418102 добавлен nltk - compound
#MAPE: 0.8741241323147318 nltk - все параметры

In [30]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [31]:
test_data.sample(10)

In [32]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [33]:
sample_submission

In [34]:
predict_submission = model.predict(test_data)

In [35]:
predict_submission

In [36]:
list(sample_submission)

In [37]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)